prepare data for MLP

In [94]:
import numpy as np
import os
import json

In [95]:
max_star = 9

In [97]:
def sort_normalize_data(data):
    normalized_data = {}
    for constellation, values in data.items():
        sorted_indices = sorted(range(len(values["area"])), key=lambda i: -values["area"][i])
        sorted_area = [values["area"][i] for i in sorted_indices]
        sorted_x = [values["x"][i] for i in sorted_indices]
        sorted_y = [values["y"][i] for i in sorted_indices]
        normalized_values = {
           "area": [(a - min(sorted_area)) / (max(sorted_area) - min(sorted_area)) for a in sorted_area],
           "x": [(x - min(sorted_x)) / (max(sorted_x) - min(sorted_x)) for x in sorted_x],
           "y": [(y - min(sorted_y)) / (max(sorted_y) - min(sorted_y)) for y in sorted_y]
       }
        normalized_data[constellation] = normalized_values
    return normalized_data

In [98]:
def create_feature_vectors(data, max_stars, target):
    feature_vectors = []
    targets = []
    for constellation, values in data.items():
        feature_vector = []
        real_attri = len(values["area"])
        for i in range(real_attri):
            for key in ["area", "x", "y"]:
                feature_vector.append(values[key][i])
        if real_attri < max_star:
            feature_vector += [0.0] *(max_star*3 - real_attri*3)
        else:
            feature_vector = feature_vector[:max_stars * 3]
        feature_vectors.append(feature_vector)
        targets.append(target)
    
    feature_vectors = np.array(feature_vectors)
    return feature_vectors, targets

In [99]:
# loop to read corrdinates and radius of stars in every constellation
input_dir = "../TrainingSample"
output_dir = "../data_model"

attributes = []# coordinates and areas
targets = []
for class_name in os.listdir(input_dir):
    if class_name in [".DS_Store", "readme.md"]:
        continue
    class_path = os.path.join(input_dir, class_name)
    starpath = os.path.join(class_path, "results.json")
    with open(starpath, 'r') as json_file:
        star_df = json.load(json_file)
    attr, target = create_feature_vectors(sort_normalize_data(star_df), max_stars=max_star, target=class_name)
    attributes.append(attr)
    targets.append(target)
attributes = np.array(attributes)
target = np.array(target)
np.save(os.path.join(output_dir, "star_coor_area_MLP.npy"),  attributes[0])
np.save(os.path.join(output_dir,"labels_MLP.npy") , targets[0])


In [100]:
star = np.load(os.path.join(output_dir, "star_coor_area_MLP.npy"))
labels = np.load(os.path.join(output_dir,"labels_MLP.npy"))
print(star.shape)
print(labels.shape)

(24, 27)
(24,)
